# Setup and data extraction

## Imports

In [ ]:
import os 
import pandas as pd
import numpy as np

from typing import Optional
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
from src.utils import load_results, get_masked_df, add_and_or_str
from src.bokeh_saving import save_figures_button
from src.mpc_dataclass import AMPC_data
from src.plotting import scatter, boxplot, get_figure_size

## Settings

In [ ]:
# Use test folder 
TEST_RESULTS: bool = False # e.g. /Results/NH_AMPC_results_Test instead of /Results/NH_AMPC_results

# Which files to use in Results folder
FILE_START_ADD: list[str] = ['ASRTID_'] # e.g. for NH_AMPC_results_ASRTID_... -> 'ASRTID_'

# Use only top n cost results
USE_TOP_N: Optional[int] = None

USE_LATEX_STYLE: bool = True

# Boxplot filter
AND_FILTER_DICT_BP: Optional[dict[str, object]] = {}
OR_FILTER_DICT_BP: Optional[dict[str, object]] = None # {'N_NN': 17}

# Scatter filter
AND_FILTER_DICT_S: Optional[dict[str, object]] = {}
OR_FILTER_DICT_S: Optional[dict[str, object]] = None   # {'N_hidden': [12, 16, 24, 32, 48]}

In [ ]:
RESULTS_DIR = os.path.abspath('Results')
AMPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'AMPC_results')
NH_AMPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'NH_AMPC_results')
SVG_RESULTS_DIR = os.path.join(RESULTS_DIR, 'SVGs')
PNG_RESULTS_DIR = os.path.join(RESULTS_DIR, 'PNGs')

if TEST_RESULTS:
    AMPC_RESULTS_DIR += '_Test'
    NH_AMPC_RESULTS_DIR += '_Test'

In [ ]:
FIGURE_SIZE_1_0 = get_figure_size(fraction=1.0) if USE_LATEX_STYLE else (1200, 800)
FIGURE_SIZE_0_8 = get_figure_size(fraction=0.8) if USE_LATEX_STYLE else (1000, 750)

## Data Extraction

### AMPC results extraction

In [ ]:
AMPC_FILE_STARTS = [f'AMPC_results_{fs_add}' for fs_add in FILE_START_ADD]

AMPC_results = []
ampc_file_paths = os.listdir(AMPC_RESULTS_DIR)
for file in ampc_file_paths:
    if not any(file.startswith(f_start) for f_start in AMPC_FILE_STARTS) or not file.endswith('.ph'):
        continue
    file_path = os.path.join(AMPC_RESULTS_DIR, file)
    results = AMPC_data.load(file_path)
    AMPC_results.append({
                        'Cost': results.Cost,
                        'Mean_Time': np.mean(results.Time) * 1e3,
                        'Median_Time': np.median(results.Time) * 1e3,
                    })
AMPC_results = pd.DataFrame(AMPC_results).mean()

### R2 Scores

In [ ]:
r2_scores_path = os.path.join(RESULTS_DIR, 'OriginalR2scores.pkl')
r2_scores = load_results(r2_scores_path)
r2_scores.head()

### Extract NH-AMPC relevant results

In [ ]:
NH_AMPC_FILE_START = [f'NH_AMPC_results_{fs_add}' for fs_add in FILE_START_ADD]

NH_AMPC_results = []
file_paths = os.listdir(NH_AMPC_RESULTS_DIR)
for file in file_paths:
    if not any(file.startswith(f_start) for f_start in NH_AMPC_FILE_START):
        continue
    file_path = os.path.join(NH_AMPC_RESULTS_DIR, file)
    results = AMPC_data.load(file_path)

    NH_AMPC_results.append({
                    'N_NN': results.P.N_NN, 
                    'N_hidden': results.P.N_hidden,
                    'acados_name': results.acados_name,
                    'Version': results.P.V_NN,
                    'Cost': results.Cost,
                    'Mean_Time': np.mean(results.Acados_Time) * 1e3,
                    'Median_Time': np.median(results.Acados_Time) * 1e3,
                    'Max_Time': np.amax(results.Acados_Time) * 1e3,
                })
    
NH_AMPC_results = pd.DataFrame(NH_AMPC_results).set_index(['N_NN', 'N_hidden', 'Version']).sort_index()

<div class="alert alert-block alert-warning">
<b>Attention:</b> Drops all failed results
</div>

In [ ]:
NH_AMPC_results.dropna(axis=0,inplace=True)
NH_AMPC_results.info()
NH_AMPC_results.head(20)

In [ ]:
NH_AMPC_combi = NH_AMPC_results.join(r2_scores, how='inner')

### Get n top cost samples  

In [ ]:
if USE_TOP_N is not None:
    indices = NH_AMPC_combi.groupby(['N_NN', 'N_hidden'], group_keys=False)['Cost'].nsmallest(n=USE_TOP_N).index
    mask = NH_AMPC_combi.index.isin(indices)
    NH_AMPC_combi = NH_AMPC_combi[mask]

### Clip cost to 150

In [ ]:
NH_AMPC_combi.loc[NH_AMPC_combi['Cost'] > 150, 'Cost'] = 150

### get mean and medians of seeds

In [ ]:
median_cost = NH_AMPC_combi.groupby(['N_NN', 'N_hidden'], group_keys=False)['Cost'].median()
mean_time_r2 = NH_AMPC_combi.groupby(['N_NN', 'N_hidden'], group_keys=False)[['Mean_Time', 'Median_Time', 'Max_Time', 'R2_score', 'Rel_err_mean', 'Rel_err_std']].median()
mm_df = pd.concat((mean_time_r2, median_cost), axis=1)
mm_df.head()

In [ ]:
df = NH_AMPC_combi.reset_index()
mm_df = mm_df.reset_index()

# Cost Scatter

## Cost - Mean_Time (legend -> N_hidden)

In [ ]:
y_label, x_label, cbar_label, legend_label = 'Cost', 'Mean_Time', 'N_NN', 'N_hidden'
filtered_df = get_masked_df(df, [], AND_FILTER_DICT_S, OR_FILTER_DICT_S)
scatter_cost_time_neurons = scatter(
    filtered_df, 
    y_label, x_label, cbar_label, legend_label,
    # title='N_MPC = 8 for all setups',
    baseline_df=AMPC_results,
    # meanmedian_df=mm_df,
    # y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_1_0,
    latex_style=USE_LATEX_STYLE,
    )
show(scatter_cost_time_neurons)

**32** neurons per hidden layer is here the best. The mean time is comparable to the samples with lower neurons per hidden layer, while the cost is still very good. Maybe 48 hidden neurons produce a better cost, but with the drawback that the mean computing time increases.

## Cost - Mean_Time (legend -> N_NN)

In [ ]:
y_label, x_label, cbar_label, legend_label = 'Cost', 'Mean_Time', 'N_hidden', 'N_NN'
filtered_df = get_masked_df(df, [], AND_FILTER_DICT_S, OR_FILTER_DICT_S)
scatter_cost_time_NH = scatter(
    filtered_df, 
    y_label, x_label, cbar_label, legend_label,
    # title='N_MPC = 8 for all setups',
    baseline_df=AMPC_results,
    # meanmedian_df=mm_df,
    # x_range=(0.02, 0.1),
    # y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_1_0,
    latex_style=USE_LATEX_STYLE,
    x_unit='ms',
    )
show(scatter_cost_time_NH)

No unknown benefit

A neural horizon of **22** is the sweetspot for this problem. The R2 score is in this setup always above 0.9 and the overall cost is the smallest. <br>
A neural horizon of **17** has nearly as good as 22 in terms of cost, whereas the R2 score is even better.

## Cost - R2_score (legend -> N_hidden)

In [ ]:
y_label, x_label, cbar_label, legend_label = 'Cost', 'R2_score', 'N_NN', 'N_hidden'
filtered_df = get_masked_df(df, [], AND_FILTER_DICT_S, OR_FILTER_DICT_S)
scatter_cost_r2_neurons = scatter(
    filtered_df, 
    y_label, x_label, cbar_label, legend_label,
    # title='N_MPC = 8 for all setups',
    baseline_df=AMPC_results,
    # meanmedian_df=mm_df,
    # y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_1_0,
    latex_style=USE_LATEX_STYLE,
    )
show(scatter_cost_r2_neurons)

## Cost - R2_score (legend -> N_NN)

In [ ]:
y_label, x_label, cbar_label, legend_label = 'Cost', 'R2_score', 'N_hidden', 'N_NN'
filtered_df = get_masked_df(df, [], AND_FILTER_DICT_S, OR_FILTER_DICT_S)
scatter_cost_r2_NH = scatter(
    filtered_df, 
    y_label, x_label, cbar_label, legend_label, 
    # title='N_MPC = 8 for all setups',
    baseline_df=AMPC_results,
    # meanmedian_df=mm_df,
    # y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_1_0,
    latex_style=USE_LATEX_STYLE,
    )
show(scatter_cost_r2_NH)

# Boxplots

### Cost

In [ ]:
x_label, y_label = 'N_NN', 'Cost'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_cost_NH = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    # title='Costs with 10 samples each',
    y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_0_8,
    scatter_colors=['darkorange' for _ in range(20)],
    latex_style=USE_LATEX_STYLE,
)
show(boxplot_cost_NH)

The neural horizon of **17** here is the sweetspot, in terms of cost. 

In [ ]:
x_label, y_label = 'N_hidden', 'Cost'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_cost_neurons = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    # title='Costs with 10 samples each',
    y_range=(103.2, 120.),
    figure_size=FIGURE_SIZE_0_8,
    scatter_colors=['darkorange' for _ in range(20)],
    latex_style=USE_LATEX_STYLE,
)
show(boxplot_cost_neurons)

Here we can see that **96** neurons per hidden layer is slightly better then **48** in terms of cost. But only if we take a look at all the data. <br>
If we look at the top 5 cost data, **48** neurons per hidden layer is the best setup, when it comes to cost.

### Mean_Time

In [ ]:
x_label, y_label = 'N_NN', 'Mean_Time'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_time_NH = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    figure_size=FIGURE_SIZE_0_8,
    # title='Mean solving time with 10 samples each',
    latex_style=USE_LATEX_STYLE,
    scatter_colors=['darkorange' for _ in range(20)],
    y_unit='ms',
)
show(boxplot_time_NH)

The computing time is slightly increasing with an increasing neural horizon. However, the neural horizon of **22** seems to make a difference here since it is slightly below 17. Nevertheless, this can also be the case due to computational inaccuracies or some other stuff. 

In [ ]:
x_label, y_label = 'N_hidden', 'Mean_Time'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_time_neurons = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    figure_size=FIGURE_SIZE_0_8,
    # title='Mean solving time with 10 samples each',
    latex_style=USE_LATEX_STYLE,
    scatter_colors=['darkorange' for _ in range(20)],
    y_unit='ms',
)
show(boxplot_time_neurons)

Smaller neurons per hidden layers are always better in terms of solving time.

### R2_score

In [ ]:
x_label, y_label = 'N_NN', 'R2_score'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_r2_NH = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    figure_size=FIGURE_SIZE_0_8,
    # title='R2 score with 10 samples each',
    latex_style=USE_LATEX_STYLE,
    scatter_colors=['darkorange' for _ in range(20)],
)
show(boxplot_r2_NH)

In [ ]:
x_label, y_label = 'N_hidden', 'R2_score'
filtered_df = get_masked_df(df, (x_label, y_label), AND_FILTER_DICT_BP, OR_FILTER_DICT_BP)
boxplot_r2_neurons = boxplot(
    filtered_df, 
    x_label, y_label,
    show_non_outliers=True, 
    show_outliers=True, 
    figure_size=FIGURE_SIZE_0_8,
    # title='R2 score with 10 samples each',
    latex_style=USE_LATEX_STYLE,
    scatter_colors=['darkorange' for _ in range(20)],
)
show(boxplot_r2_neurons)

# Save Plots

In [ ]:
all_plots = [
    ('scatter_cost_time_NH', scatter_cost_time_NH.children), 
    ('scatter_cost_time_neurons', scatter_cost_time_neurons.children),
    ('scatter_cost_r2_NH', scatter_cost_r2_NH.children),
    ('scatter_cost_r2_neurons', scatter_cost_r2_neurons.children),
    ('boxplot_cost_NH', boxplot_cost_NH),
    ('boxplot_cost_neurons', boxplot_cost_neurons),
    ('boxplot_time_NH', boxplot_time_NH),
    ('boxplot_time_neurons', boxplot_time_neurons),
    ('boxplot_r2_NH', boxplot_r2_NH),
    ('boxplot_r2_neurons', boxplot_r2_neurons),
]

for i, (name, p) in enumerate(all_plots):
    if 'scatter' in name:
        name = add_and_or_str(name, OR_FILTER_DICT_S, AND_FILTER_DICT_S)
    elif 'boxplot' in name:
        name = add_and_or_str(name, OR_FILTER_DICT_BP, AND_FILTER_DICT_BP)
    
    all_plots[i] = (name, p)

save_figures_button(all_plots, SVG_RESULTS_DIR, PNG_RESULTS_DIR)